In [1]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import neuPrecLoss
from misc.loader import RecDataset
import models
from models.loss import *

In [3]:
with open(os.path.join("data", "parsed", "ml-1m-new"), 'rb') as f:
    a,b,c, tr_tr, tr_val, te = pickle.load(f)

ValueError: too many values to unpack (expected 6)

In [ ]:
import pickle
best_model = torch.load("saved_models/ml-20m/vae_prec5")

In [ ]:
best_model

In [30]:
ae_dataset = AEDataset(tr_tr)
best_model

{'model': MultiVAE(
   (q_layers): ModuleList(
     (0): Linear(in_features=13681, out_features=256, bias=True)
     (1): Linear(in_features=256, out_features=1024, bias=True)
   )
   (p_layers): ModuleList(
     (0): Linear(in_features=512, out_features=256, bias=True)
     (1): Linear(in_features=256, out_features=13681, bias=True)
   )
   (drop): Dropout(p=0.8, inplace=False)
 ),
 'epoch': 5,
 'bs': 512,
 'lr': 0.001,
 'dim': [512, 256],
 'lamb': 0.0001,
 'metric_5': {'precision': 0.08936000000000194,
  'recall': 0.05921655429419089,
  'map': 0.1920347222222216,
  'ndcg': 0.1088772340564977},
 'metric_50': {'precision': 0.027200000000000835,
  'recall': 0.1565233167054972,
  'map': 0.160146862163632,
  'ndcg': 0.1181003027475717},
 'dropout': 0.8}

In [25]:
print(ranking_metrics_at_k(wrapper, tr_tr, te, K=5, num_threads=4))
print(ranking_metrics_at_k(wrapper, tr_tr, te, K=10, num_threads=4))


/home/ita/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


{'precision': 0.0869800000000019, 'recall': 0.058693894298560416, 'map': 0.1896362499999995, 'ndcg': 0.10652404038939785}
{'precision': 0.05758000000000196, 'recall': 0.07443248352015566, 'map': 0.188871258314436, 'ndcg': 0.09663626218533948}


In [32]:
from models.lrloss import approxNDCGLoss, rankNet, lambdaLoss

In [34]:
n_users, n_items = tr_tr.shape

In [38]:
import models.mf
import neuralsort.neuralobjs
importlib.reload(models.ae)
loader = DataLoader(ae_dataset, batch_size=128, shuffle=True, num_workers=6)
model = models.ae.MultiVAE(best_model['dim'] + [n_items], dropout=best_model['dropout'])
model.load_state_dict(bm_state)
model.cuda()
optimizer = torch.optim.RMSprop(model.parameters(), lr=1 * 1e-6, weight_decay=0)
sc = neuralsort.neuralobjs.SC()
lm = -1 
for epoch in (range(1, 10 + 1)):
    tot_loss = 0
    tot_loss = 0
    model = model.train()
    for uid, rowl in tqdm(loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        scores, _, _ = model.forward(row)
        loss  = neuPrecLoss(sc, scores, row, topk=300, k=3, tau=20.0, use_top=True).mean()
        loss.backward()
        tot_loss += loss.detach().cpu().numpy()
        optimizer.step()
    model = model.eval()
    if (epoch % 1 == 0):
        wrapper = models.ae.implicitWrapper(model, tr_tr, naive_sparse2tensor, vae=True)
        mm = (ranking_metrics_at_k(wrapper, tr_tr, tr_val, K=5, num_threads=4))
        print(ranking_metrics_at_k(wrapper, tr_tr, te, K=5, num_threads=4))
        print(ranking_metrics_at_k(wrapper, tr_tr, te, K=10, num_threads=4))
        if mm['precision'] < lm:
            break
        lm =  mm['precision']
        


{'precision': 0.09252000000000182, 'recall': 0.06893234585367138, 'map': 0.1911479166666667, 'ndcg': 0.1124682288678667}
{'precision': 0.05954000000000168, 'recall': 0.09050383389671461, 'map': 0.18771905083144333, 'ndcg': 0.10345798756526467}



{'precision': 0.07938000000000113, 'recall': 0.061483804651446636, 'map': 0.17216472222222182, 'ndcg': 0.0984565852346703}
{'precision': 0.05406000000000138, 'recall': 0.08363424381970325, 'map': 0.16756606547619002, 'ndcg': 0.09430571414912392}


In [35]:
import models.mf
import neuralsort.neuralobjs
importlib.reload(models.ae)
loader = DataLoader(ae_dataset, batch_size=128, shuffle=True, num_workers=6)
model = models.ae.MultiVAE(best_model['dim'] + [n_items], dropout=best_model['dropout'])
model.load_state_dict(bm_state)
model.cuda()
optimizer = torch.optim.RMSprop(model.parameters(), lr=1 * 1e-6, weight_decay=0)
sc = neuralsort.neuralobjs.SC()
lm = -1 
for epoch in (range(1, 10 + 1)):
    tot_loss = 0
    tot_loss = 0
    model = model.train()
    for uid, rowl in tqdm(loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        scores, _, _ = model.forward(row)
        y_hat, indices = torch.topk(scores, 300)
        y_true = row.gather(1, indices)
        loss  = approxNDCGLoss(y_hat, y_true)
        loss.backward()
        tot_loss += loss.detach().cpu().numpy()
        optimizer.step()
    model = model.eval()
    if (epoch % 1 == 0):
        wrapper = models.ae.implicitWrapper(model, tr_tr, naive_sparse2tensor, vae=True)
        mm = (ranking_metrics_at_k(wrapper, tr_tr, tr_val, K=5, num_threads=4))
        print(ranking_metrics_at_k(wrapper, tr_tr, te, K=5, num_threads=4))
        print(ranking_metrics_at_k(wrapper, tr_tr, te, K=10, num_threads=4))
        if mm['precision'] < lm:
            break
        lm =  mm['precision']
        


{'precision': 0.08726000000000181, 'recall': 0.058231955211608614, 'map': 0.1922705555555552, 'ndcg': 0.1079217174581876}
{'precision': 0.06052000000000179, 'recall': 0.08518413458590629, 'map': 0.1874104445389255, 'ndcg': 0.10186313399251082}



{'precision': 0.08378000000000142, 'recall': 0.05604487086196955, 'map': 0.18237944444444443, 'ndcg': 0.10359700904072658}
{'precision': 0.058620000000001504, 'recall': 0.08333295779065585, 'map': 0.17737412443310613, 'ndcg': 0.09843591113248824}


In [36]:
import models.mf
import neuralsort.neuralobjs
importlib.reload(models.ae)
loader = DataLoader(ae_dataset, batch_size=128, shuffle=True, num_workers=6)
model = models.ae.MultiVAE(best_model['dim'] + [n_items], dropout=best_model['dropout'])
model.load_state_dict(bm_state)
model.cuda()
optimizer = torch.optim.RMSprop(model.parameters(), lr=1 * 1e-6, weight_decay=0)
sc = neuralsort.neuralobjs.SC()
lm = -1 
for epoch in (range(1, 10 + 1)):
    tot_loss = 0
    tot_loss = 0
    model = model.train()
    for uid, rowl in tqdm(loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        scores, _, _ = model.forward(row)
        y_hat, indices = torch.topk(scores, 300)
        y_true = row.gather(1, indices)
        loss  = rankNet(y_hat, y_true)
        loss.backward()
        tot_loss += loss.detach().cpu().numpy()
        optimizer.step()
    model = model.eval()
    if (epoch % 1 == 0):
        wrapper = models.ae.implicitWrapper(model, tr_tr, naive_sparse2tensor, vae=True)
        mm = (ranking_metrics_at_k(wrapper, tr_tr, tr_val, K=5, num_threads=4))
        print(ranking_metrics_at_k(wrapper, tr_tr, te, K=5, num_threads=4))
        print(ranking_metrics_at_k(wrapper, tr_tr, te, K=10, num_threads=4))
        if mm['precision'] < lm:
            break
        lm =  mm['precision']
        


{'precision': 0.059979999999999555, 'recall': 0.04756488722479732, 'map': 0.1309993055555562, 'ndcg': 0.07296570766596301}
{'precision': 0.052570000000001144, 'recall': 0.07949942021948003, 'map': 0.1318094501606197, 'ndcg': 0.08096075060319999}



{'precision': 0.04511999999999891, 'recall': 0.038313029695334504, 'map': 0.09720777777777867, 'ndcg': 0.055102276956149956}
{'precision': 0.04111000000000043, 'recall': 0.06634588440555968, 'map': 0.09559109646636457, 'ndcg': 0.06372786489989042}


In [37]:
import models.mf
import neuralsort.neuralobjs
importlib.reload(models.ae)
loader = DataLoader(ae_dataset, batch_size=128, shuffle=True, num_workers=6)
model = models.ae.MultiVAE(best_model['dim'] + [n_items], dropout=best_model['dropout'])
model.load_state_dict(bm_state)
model.cuda()
optimizer = torch.optim.RMSprop(model.parameters(), lr=1 * 1e-6, weight_decay=0)
sc = neuralsort.neuralobjs.SC()
lm = -1 
for epoch in (range(1, 10 + 1)):
    tot_loss = 0
    tot_loss = 0
    model = model.train()
    for uid, rowl in tqdm(loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        scores, _, _ = model.forward(row)
        y_hat, indices = torch.topk(scores, 300)
        y_true = row.gather(1, indices)
        loss  = lambdaLoss(y_hat, y_true)
        loss.backward()
        tot_loss += loss.detach().cpu().numpy()
        optimizer.step()
    model = model.eval()
    if (epoch % 1 == 0):
        wrapper = models.ae.implicitWrapper(model, tr_tr, naive_sparse2tensor, vae=True)
        mm = (ranking_metrics_at_k(wrapper, tr_tr, tr_val, K=5, num_threads=4))
        print(ranking_metrics_at_k(wrapper, tr_tr, te, K=5, num_threads=4))
        print(ranking_metrics_at_k(wrapper, tr_tr, te, K=10, num_threads=4))
        if mm['precision'] < lm:
            break
        lm =  mm['precision']
        


{'precision': 0.0599999999999996, 'recall': 0.04766169124615044, 'map': 0.13136166666666713, 'ndcg': 0.07319896417089926}
{'precision': 0.05296000000000114, 'recall': 0.07939386103225726, 'map': 0.13342043164210082, 'ndcg': 0.08132935090919093}


KeyboardInterrupt: 